In [29]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import math
import re
import pickle
from sklearn import preprocessing
from sklearn import tree
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.decomposition import PCA
# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
# Cost Functions
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics

# Preprocessing
from sklearn.impute import SimpleImputer

# Scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

# Feature Engineering
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
# This is for regression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# This is for classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [12]:
# Load the dataset
df=pd.read_csv("/content/drive/MyDrive/Capstone Project- Track Three/US_accidents.csv")
df.head()

,ID,Severity,Month,Year,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Street,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2,2016,40.108910,-83.092860,40.112060,-83.031870,3.230,Outerbelt E,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2,2016,39.865420,-84.062800,39.865010,-84.048730,0.747,I-70 E,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2,2016,39.102660,-84.524680,39.102090,-84.523960,0.055,I-75 S,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2,2016,41.062130,-81.537840,41.062170,-81.535470,0.123,I-77 N,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2,2016,39.172393,-84.492792,39.170476,-84.501798,0.500,I-75 S,...,False,False,False,False,False,False,Day,Day,Day,Day


In [13]:
print('Shape of dataset is ', df.shape)

Shape of dataset is  (1048575, 44)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 44 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID                     1048575 non-null  object 
 1   Severity               1048575 non-null  int64  
 2   Month                  1048575 non-null  int64  
 3   Year                   1048575 non-null  int64  
 4   Start_Lat              1048575 non-null  float64
 5   Start_Lng              1048575 non-null  float64
 6   End_Lat                1048575 non-null  float64
 7   End_Lng                1048575 non-null  float64
 8   Distance(mi)           1048575 non-null  float64
 9   Street                 1048574 non-null  object 
 10  Side                   1048575 non-null  object 
 11  City                   1048520 non-null  object 
 12  County                 1048575 non-null  object 
 13  State                  1048575 non-null  object 
 14  Country           

In [15]:
df.isna().sum()

ID                            0
Severity                      0
Month                         0
Year                          0
Start_Lat                     0
Start_Lng                     0
End_Lat                       0
End_Lng                       0
Distance(mi)                  0
Street                        1
Side                          0
City                         55
County                        0
State                         0
Country                       0
Timezone                   1026
Airport_Code               3647
Weather_Timestamp         15516
Temperature(F)            21746
Wind_Chill(F)            214122
Humidity(%)               22728
Pressure(in)              18585
Visibility(mi)            22534
Wind_Direction            22626
Wind_Speed(mph)           57687
Precipitation(in)        231683
Weather_Condition         22892
Amenity                       0
Bump                          0
Crossing                      0
Give_Way                      0
Junction

In [16]:
def missing (data):
    missing_number = data.isnull().sum().sort_values(ascending=False)
    missing_percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent*100], axis=1, keys=['Missing_Number', 'Missing_Percent %'])
    return missing_values

missing(df)

,Missing_Number,Missing_Percent %
Precipitation(in),231683,22.095034
Wind_Chill(F),214122,20.420285
Wind_Speed(mph),57687,5.501466
Weather_Condition,22892,2.183153
Humidity(%),22728,2.167513
Wind_Direction,22626,2.157786
Visibility(mi),22534,2.149012
Temperature(F),21746,2.073862
Pressure(in),18585,1.772405
Weather_Timestamp,15516,1.479722


In [17]:
# Fill in the Missing Values using the Simple Imputer with the mean strategy
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(df[['Precipitation(in)', 'Wind_Chill(F)']])
df[['Precipitation(in)', 'Wind_Chill(F)']] = imputer.transform(df[['Precipitation(in)', 'Wind_Chill(F)']])

In [18]:
df.dropna(axis=0,inplace=True)

In [19]:
# Re-check the missing values.
df.isna().sum()

ID                       0
Severity                 0
Month                    0
Year                     0
Start_Lat                0
Start_Lng                0
End_Lat                  0
End_Lng                  0
Distance(mi)             0
Street                   0
Side                     0
City                     0
County                   0
State                    0
Country                  0
Timezone                 0
Airport_Code             0
Weather_Timestamp        0
Temperature(F)           0
Wind_Chill(F)            0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
T

In [20]:
print('Shape of dataset is ', df.shape)

Shape of dataset is  (978793, 44)


In [21]:
#Dropping unnessacary columns
column=["ID","City","County","Country","Timezone","Weather_Timestamp","Airport_Code"]
df.drop(columns=column, inplace=True)

In [22]:
print('Shape of dataset is ', df.shape)

Shape of dataset is  (978793, 37)


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978793 entries, 0 to 1048574
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Severity               978793 non-null  int64  
 1   Month                  978793 non-null  int64  
 2   Year                   978793 non-null  int64  
 3   Start_Lat              978793 non-null  float64
 4   Start_Lng              978793 non-null  float64
 5   End_Lat                978793 non-null  float64
 6   End_Lng                978793 non-null  float64
 7   Distance(mi)           978793 non-null  float64
 8   Street                 978793 non-null  object 
 9   Side                   978793 non-null  object 
 10  State                  978793 non-null  object 
 11  Temperature(F)         978793 non-null  float64
 12  Wind_Chill(F)          978793 non-null  float64
 13  Humidity(%)            978793 non-null  float64
 14  Pressure(in)           978793 non-n

In [27]:
#Encoding our categorical columns
catList = df.select_dtypes(include = ["object","bool"]).columns

In [28]:
print (catList)

Index(['Street', 'Side', 'State', 'Wind_Direction', 'Weather_Condition',
       'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
       'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming',
       'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')


In [30]:
le = LabelEncoder()

for feat in catList:
    df[feat] = le.fit_transform(df[feat].astype(str))

print (df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978793 entries, 0 to 1048574
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Severity               978793 non-null  int64  
 1   Month                  978793 non-null  int64  
 2   Year                   978793 non-null  int64  
 3   Start_Lat              978793 non-null  float64
 4   Start_Lng              978793 non-null  float64
 5   End_Lat                978793 non-null  float64
 6   End_Lng                978793 non-null  float64
 7   Distance(mi)           978793 non-null  float64
 8   Street                 978793 non-null  int64  
 9   Side                   978793 non-null  int64  
 10  State                  978793 non-null  int64  
 11  Temperature(F)         978793 non-null  float64
 12  Wind_Chill(F)          978793 non-null  float64
 13  Humidity(%)            978793 non-null  float64
 14  Pressure(in)           978793 non-n